In [9]:
import bokeh
import pandas as pd
import json

In [108]:
from lxml import html
from collections import defaultdict
import requests
import json

def from_html_to_json(url):

    requests_session = requests.session()
    page = requests_session.get(url)

    html_tree = html.fromstring(page.content)

    if 'www.fio.cz' in url:

        structure = ('date', 'amount', 'type', 'name', 'messege', 'KS', 'VS', 'SS', 'note')

        transactions = defaultdict(lambda: {'date': '',
                                            'amount': '',
                                            'type': '',
                                            'name': '',
                                            'messege': '',
                                            'KS': '',
                                            'VS': '',
                                            'SS': '',
                                            'note': ''
                                            })

        # //table/tbody/tr/td[1]/text()

        for tr in enumerate(html_tree.xpath('//table/tbody/tr')[1:]):
            for piece in enumerate(structure):
                if piece[0] == 1:
                    transactions[tr[0]][piece[1]] = float(tr[1].xpath('td[2]/@data-value')[0])
                else:
                    try:
                        transactions[tr[0]][piece[1]] = tr[1].xpath('td[{}]/text()'.format(piece[0]+1))[0]
                    except IndexError:
                        pass

        return json.dumps(transactions)

#from_html_to_json('https://www.fio.cz/ib2/transparent?a=2600088789')

print(from_html_to_json('https://www.fio.cz/ib2/transparent?a=2600088789'))

{"0": {"messege": "\n\t\t", "date": "15.06.2017", "SS": "", "VS": "7511041582", "note": "Martin Liter\u00e1k", "name": "Martin Liter\u00e1k", "type": "Bezhotovostn\u00ed p\u0159\u00edjem", "KS": "0000", "amount": 500.0}, "1": {"messege": "\n\t\t\n\t\t\n\t\t\n\t\t\n\t\t\n\t", "date": "15.06.2017", "SS": "", "VS": "", "note": "Tesa\u0159 Tom\u00e1\u0161", "name": "Tesa\u0159 Tom\u00e1\u0161", "type": "Bezhotovostn\u00ed p\u0159\u00edjem", "KS": "", "amount": 200.0}, "2": {"messege": "\n\t\t\n\t\t\n\t\t\n\t\t\n\t\t\n\t", "date": "15.06.2017", "SS": "", "VS": "", "note": "Michali\u010dka J\u00e1n,Mgr.", "name": "Michali\u010dka J\u00e1n,Mgr.", "type": "Bezhotovostn\u00ed p\u0159\u00edjem", "KS": "", "amount": 100.0}, "3": {"messege": "\n\t\t\n\t\t\n\t\t\n\t\t\n\t\t\n\t", "date": "15.06.2017", "SS": "", "VS": "120600", "note": "Novotn\u00e1 Marie", "name": "Novotn\u00e1 Marie", "type": "Bezhotovostn\u00ed p\u0159\u00edjem", "KS": "", "amount": 100.0}, "4": {"messege": "\n\t\t\n\t\t\n\t\t\n\

In [109]:
import parse_html

{"0": {"messege": "\n\t\t", "date": "15.06.2017", "SS": "", "VS": "7511041582", "note": "Martin Liter\u00e1k", "name": "Martin Liter\u00e1k", "type": "Bezhotovostn\u00ed p\u0159\u00edjem", "KS": "0000", "amount": 500.0}, "1": {"messege": "\n\t\t\n\t\t\n\t\t\n\t\t\n\t\t\n\t", "date": "15.06.2017", "SS": "", "VS": "", "note": "Tesa\u0159 Tom\u00e1\u0161", "name": "Tesa\u0159 Tom\u00e1\u0161", "type": "Bezhotovostn\u00ed p\u0159\u00edjem", "KS": "", "amount": 200.0}, "2": {"messege": "\n\t\t\n\t\t\n\t\t\n\t\t\n\t\t\n\t", "date": "15.06.2017", "SS": "", "VS": "", "note": "Michali\u010dka J\u00e1n,Mgr.", "name": "Michali\u010dka J\u00e1n,Mgr.", "type": "Bezhotovostn\u00ed p\u0159\u00edjem", "KS": "", "amount": 100.0}, "3": {"messege": "\n\t\t\n\t\t\n\t\t\n\t\t\n\t\t\n\t", "date": "15.06.2017", "SS": "", "VS": "120600", "note": "Novotn\u00e1 Marie", "name": "Novotn\u00e1 Marie", "type": "Bezhotovostn\u00ed p\u0159\u00edjem", "KS": "", "amount": 100.0}, "4": {"messege": "\n\t\t\n\t\t\n\t\t\n\

In [107]:
from pandas.io.json import json_normalize
d = from_html_to_json('https://www.fio.cz/ib2/transparent?a=2600088789')
data = json.loads(d)
json_normalize(data)

,0.KS,0.SS,0.VS,0.amount,0.date,0.messege,0.name,0.note,0.type,1.KS,...,8.type,9.KS,9.SS,9.VS,9.amount,9.date,9.messege,9.name,9.note,9.type
0,0000,,7511041582,500.0,15.06.2017,\n\t\t,Martin Literák,Martin Literák,Bezhotovostní příjem,,...,Bezhotovostní příjem,,,11501,222.0,02.06.2017,\n\t\t,Bučková Bronislava,Bučková Bronislava,Bezhotovostní příjem


In [91]:
df = from_html_to_csv('https://www.fio.cz/ib2/transparent?a=2600088789')
df.xpath('//*/tbody/tr[3]/td[2]')[0].text

/home/zuzi/hackathon/2017/hekac_2017/venv/lib/python3.5/site-packages/ipykernel_launcher.py:35: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.



'\n\t\t100,00\xa0CZK\n\t'

In [93]:
df = from_html_to_csv('https://www.fio.cz/ib2/transparent?a=2600088789')
df.xpath('//*/tbody/tr[5]/td[4]')[0].text

/home/zuzi/hackathon/2017/hekac_2017/venv/lib/python3.5/site-packages/ipykernel_launcher.py:35: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.



'Radek Malenovský'

In [88]:
df = from_html_to_csv('https://www.fio.cz/ib2/transparent?a=2600088789')
df

/home/zuzi/hackathon/2017/hekac_2017/venv/lib/python3.5/site-packages/ipykernel_launcher.py:35: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.



0           1           2           3           4   \
major minor                                                                 
0     KS                                                                    
      SS                                                                    
      VS                                                                    
      amount                                                                
      date     15.06.2017  15.06.2017  15.06.2017  15.06.2017  12.06.2017   
      messege                                                               
      name                                                                  
      note                                                                  
      type                                                                  

                       5           6           7           8           9   \
major minor                                                                 
0     KS                                                                    
      SS                                                                    
      VS                                                                    
      amount                                                                
      date     12.06.2017  07.06.2017  07.06.2017  05.06.2017  02.06.2017   
      messege                                                               
      name                                                                  
      note                                                                  
      type                                                                  

                       10  
major minor                
0     KS                   
      SS                   
      VS                   
      amount               
      date     19.05.2017  
      messege              
      name                 
      note                 
      type